In [3]:
import plotly.graph_objects as go
from scipy.stats import norm
import math
import numpy as np

# Valuation

## Estimation of the cost of equity

## How to de-/re-leverage a beta

## Derivation of delevered (or unlevered) beta:

## WACC

In [4]:
# CAPM
def calculate_re(rf, beta, rm):
    '''
    Calculates re, the cost of an equity
    Parameters
    ----------
    rf: float
    risk-free rate
    beta: float
    beta of the stock (market risk)
    rm: float
    market return
    Notes
    -----
    rm-rf is the premium of the market over the risk-free rate
    '''
    re = rf + beta * (rm - rf)
    return re


# Hamada-Equation, supposing beta_debt=0
def hamada(beta_u, Vd, Ve, tau):
    # beta_u: unlevered beta
    # Vd: debt
    # Ve: equity
    # tau: tax rate
    return beta_u * (1 + Vd / Ve * (1 - tau))


def hamada_(beta_e, Vd, Ve, tau):
    # beta_e: levered beta
    # Vd: debt
    # Ve: equity
    # tau: tax rate

    return beta_e * (1 / (1 + Vd / Ve * (1 - tau)))


def calculate_wacc(ke, kd, Ve, Vd, tau):
    V = Ve + Vd
    return ke * Ve / V + kd * (1 - tau) * Vd / V



In [2]:
hamada(.8, 9, 6, .3)

1.64

In [5]:
# 1. Calculate the unlevered beta
Vd_comp = 0.2
Ve_comp = 7.1

# debt_equity_ratio = 0.8/2.3
# 
# if not Ve_comp:
#     Ve_comp = Vd_comp/debt_equity_ratio
# else:
#     Vd_comp = Ve_comp*debt_equity_ratio

tau_comp = .33
beta_e_comp = 1.24
beta_u_comp = hamada_(beta_e_comp, Vd_comp, Ve_comp, tau_comp)
print(f'The unlevered beta is {beta_u_comp:.2f}')

# We assume beta_u_comp = beta_u
beta_u = beta_u_comp

# 2. Calculate the relevered beta
Vd = 400
Ve = 11.5

debt_equity_ratio = 0.8 / 2.3

if not Ve:
    Ve = Vd / debt_equity_ratio
else:
    Vd = Ve * debt_equity_ratio

tau = .33
beta = hamada(beta_u, Vd, Ve, tau)

print(f'The relevered beta is {beta:.2f}')

# 3. Calculate the cost of equity using the relevered beta and CAPM
# cost of equity
rf = .076
rm = .121
re = calculate_re(rf, beta, rm)

print(f'The cost of equity is {re * 100:.2f}%')

# 4. Calculate the cost of debt
ke = re
kd = .1
Ve = 1150
Vd = 400
WACC = calculate_wacc(ke, kd, Ve, Vd, tau)

print(f'The WACC is {WACC * 100:.2f}%')

The unlevered beta is 1.22
The relevered beta is 1.50
The cost of equity is 14.35%
The WACC is 12.38%


## Present Value (PV)

In [6]:
def calculate_PV(cashflows, rate):
    '''
    Calculates the Present Value of a company based on the projection of the cashflow
    cashflows: list[float]
    rate: float
    constant rate utilized, can be WACC
    '''
    PV = sum([cf / ((1 + rate) ** (i + 1)) for i, cf in enumerate(cashflows)])
    return PV

def calculate_PV_perpetuity(cashflow_t, rate, g, t):
    '''
    Calculates the Present Value of a perpetuity based on the projection of the cashflow
    cashflow_t: float
    cashflow at time t
    rate: float
    constant rate utilized, can be WACC or cost of equity (k_u)
    g: float
    growth rate
    t: int
    number of periods
    '''
    PV = cashflow_t * (1 + g) / ((rate - g) * (1 + rate) ** t) 
    return PV

def calculate_PV_interest_tax_shields(interest_expenses, rate, tau):
    '''
    Calculates the Present Value of the interest tax shields
    interest_expenses: list[float]
    rate: float
    constant rate utilized, can be WACC or cost of debt (k_d)
    tau: float
    tax rate (0 < tau < 1)
    '''
    PV = sum([interest * tau / ((1 + rate) ** (i + 1)) for i, interest in enumerate(interest_expenses)])
    return PV

def calculate_PV_tax_shields_perpetuity(FCFF, g, WACC, n):
    '''
    Calculates the Present Value of the tax shields beyond a certain year in perpetuity
    FCFF: float
        Free Cash Flow to the Firm at the final projected year
    g: float
        Growth rate of the cash flows
    WACC: float
        Weighted Average Cost of Capital
    n: int
        Number of years till the final projected year
    '''
    TVL = (FCFF * (1 + g)) / (WACC - g)
    PV_TS = TVL / ((1 + WACC) ** n)
    return PV_TS

def calculate_firm_value(V_U, PV_TS):
    '''
    Calculates the total value of the levered firm
    V_U: float
        Value of the unlevered firm
    PV_TS: float
        Present Value of the Tax Shields
    '''
    V_L = V_U + PV_TS
    return V_L

def calculate_equity_value(V_L, V_D, shares_outstanding):
    '''
    Calculates the equity value and value per share
    V_L: float
        Value of the levered firm
    V_D: float
        Value of the firm's debt
    shares_outstanding: float
        Number of shares outstanding
    '''
    V_E = V_L - V_D
    value_per_share = V_E / shares_outstanding
    return V_E, value_per_share

In [7]:
cashflows = [-397.6, 156.0, 32.9, 176.9, 315.2, 400.4, 448.8, 436.8, 484, 498.4]
rate = 0.1238  # from last code-cell (WACC)
PV = calculate_PV(cashflows, rate)
print(f"The present value of the company is {PV:.2f}")


The present value of the company is 972.82


In [8]:
def get_entity_value_dcf(cashflows, rate, g, pv_debt=None):
    # we approximate the cashflow in t+1 with the value at t with growth g and rate
    terminal_value = cashflows[-1] * (1 + g) / (rate - g)

    # get it to present value
    PV_final = terminal_value / (1 + rate) ** (len(cashflows))
    PV = calculate_PV(cashflows, rate)
    if pv_debt:
        V = PV + PV_final - pv_debt
    else:
        V = PV + PV_final

    return {
        'terminal_value': terminal_value,
        'PV_final': PV_final,
        'PV': PV,
        'V': V
    }

In [9]:
cashflows = [-397.6, 156.0, 32.9, 176.9, 315.2, 400.4, 448.8, 436.8, 484, 498.4]
# g - growth rate
g = .04
# re - cost of equity
re = .1435
# rd - cost of debt
rd = .1
# Vd - debt
Vd = 400

debt_equity_ratio = 0.8 / 2.3
# Ve - equity
Ve = None

if not Ve:
    Ve = Vd / debt_equity_ratio
else:
    Vd = Ve * debt_equity_ratio

tau = .33

WACC = calculate_wacc(re, rd, Ve, Vd, tau)

rate = WACC
result = get_entity_value_dcf(cashflows, rate, g, Vd)

print(f"The Enterprise Value (Terminal Value) is {result['terminal_value']:.2f}")
print(f"The Present Value of the Terminal Value is {result['PV_final']:.2f}")
print(f"The Present Value of the company (equity) is {result['V']:.2f}")



The Enterprise Value (Terminal Value) is 6188.49
The Present Value of the Terminal Value is 1926.89
The Present Value of the company (equity) is 2500.02


## Specific APV cases

In [10]:
def calculate_apv(re, rd, t, rf, rm, k, Vd=None, Ve=None):
    '''
    k=0: case I (constant leverage ratio) => rt = ru
    k=1: case II (constant debt level) => rt = rd
    '''
    if Vd:
        Ve = 1 - Vd
    else:
        Vd = 1 - Ve

    ru = rd * Vd + re * Ve
    WACC = ru - Vd * t * (rd + k * (ru - rd))
    betau = (ru - rf) / rm
    betae = (re - rf) / rm
    betad = (betau - Ve * betae) / Vd

    return {
        'ru': ru,
        'WACC': WACC,
        'betau': betau,
        'betae': betae,
        'betad': betad
    }


In [11]:
# re - cost of equity
re = 0.1
# rd - cost of debt
rd = 0.06
# t - tax rate
t = 0.35
# rf - risk-free rate
rf = 0.02
# rm - market return
rm = 0.06
# k - case (0=constant leverage ratio, 1=constant debt level)
k = 1  # 

result = calculate_apv(re, rd, t, rf, rm, k, Vd=.5)
print(f"The unlevered cost of equity is {result['ru']:.2f}")
print(f"The WACC is {result['WACC'] * 100:.2f}%")
print(f"The unlevered beta is {result['betau']:.2f}")
print(f"The levered beta is {result['betae']:.2f}")
print(f"The beta of the debt is {result['betad']:.2f}")


The unlevered cost of equity is 0.08
The WACC is 6.60%
The unlevered beta is 1.00
The levered beta is 1.33
The beta of the debt is 0.67


## Multiple Valuation


In [30]:
def calculate_multiples(price, revenue, earnings, ebit, ebitda, book_value, debt, num_shares):
    multiples = {
        'P/Rev': price / revenue * num_shares,
        'P/E': price / earnings * num_shares,
        'P/B': price / book_value * num_shares,
        'EV/EBIT': (price * num_shares + debt) / ebit,
        'EV/EBITDA': (price * num_shares + debt) / ebitda
    }
    return multiples


def calculate_mean_min_max_multiples(multiples_list):
    mean_multiples = {}
    min_multiples = {}
    max_multiples = {}
    
    for multiples in multiples_list:
        for key, value in multiples.items():
            if key not in mean_multiples:
                mean_multiples[key] = value
                min_multiples[key] = value
                max_multiples[key] = value
            else:
                mean_multiples[key] += value
                min_multiples[key] = min(min_multiples[key], value)
                max_multiples[key] = max(max_multiples[key], value)
                
    for key in mean_multiples:
        mean_multiples[key] /= len(multiples_list)
        
    return mean_multiples, min_multiples, max_multiples


def calculate_estimated_prices_at_multiples(multiples, price, revenue, earnings, ebit, ebitda, book_value, debt, num_shares):
    estimated_prices = {
        'P/Rev': max(0, multiples['P/Rev'] * revenue / num_shares),
        'P/E': max(0, multiples['P/E'] * earnings / num_shares),
        'P/B': multiples['P/B'] * book_value / num_shares,
        'EV/EBIT': (multiples['EV/EBIT'] * ebit - debt) / num_shares,
        'EV/EBITDA': (multiples['EV/EBITDA'] * ebitda - debt) / num_shares
    }
    return estimated_prices


def calculate_estimated_prices_range(mean_multiples, min_multiples, max_multiples, target_data):
    mean_prices = calculate_estimated_prices_at_multiples(mean_multiples, **target_data)
    min_prices = calculate_estimated_prices_at_multiples(min_multiples, **target_data)
    max_prices = calculate_estimated_prices_at_multiples(max_multiples, **target_data)
    
    return mean_prices, min_prices, max_prices


def estimate_stock_price_from_range(estimated_prices):
    mean_price = sum(estimated_prices[0].values()) / len(estimated_prices[0])
    min_price = sum(estimated_prices[1].values()) / len(estimated_prices[1])
    max_price = sum(estimated_prices[2].values()) / len(estimated_prices[2])
    
    return min_price, mean_price, max_price


def print_multiples(multiples, label):
    print(f"\n{label} Multiples:")
    for key, value in multiples.items():
        print(f"{key}: {value:.2f}")


def print_estimated_prices(estimated_prices, label):
    print(f"\n{label} Prices for Target Company:")
    for key, value in estimated_prices.items():
        print(f"{key}: {value:.2f}")


def print_estimated_stock_price_range(min_price, mean_price, max_price):
    print('\n' + '=' * 50)
    print(f"Estimated Stock Price for Target Company:")
    print(f"Min: {min_price:.2f}, Mean: {mean_price:.2f}, Max: {max_price:.2f}")
    print('=' * 50)

In [31]:
companies_data = {
    'company1': {
        'price': 60.91,
        'revenue': 129.872,
        'earnings': 9.272,
        'ebit': 9.346,
        'ebitda': 13.04,
        'book_value': 43.665,
        'debt': 86.689,
        'num_shares': 1.0698
    },
    'company2': {
        'price': 70.09,
        'revenue': 80.401,
        'earnings': 6.368,
        'ebit': 8.99,
        'ebitda': 12.092,
        'book_value': 37.22,
        'debt': 77.506,
        'num_shares': 0.6565
    },
    'target_company': {
        'price': 95.2,
        'revenue': 202.458,
        'earnings': 8.896,
        'ebit': 11.921,
        'ebitda': 20.505,
        'book_value': 89.991,
        'debt': 108.647,
        'num_shares': 0.5013
    }
}

# Calculate multiples for company1 and company2
company1_multiples = calculate_multiples(**companies_data['company1'])
company2_multiples = calculate_multiples(**companies_data['company2'])

# Print multiples for company1 and company2
print_multiples(company1_multiples, "Company 1")
print_multiples(company2_multiples, "Company 2")

# Calculate mean, min, and max multiples
mean_multiples, min_multiples, max_multiples = calculate_mean_min_max_multiples([company1_multiples, company2_multiples])

# Print mean, min, and max multiples
print_multiples(mean_multiples, "Mean")
print_multiples(min_multiples, "Min")
print_multiples(max_multiples, "Max")

# Estimate stock prices for target company
mean_prices, min_prices, max_prices = calculate_estimated_prices_range(mean_multiples, min_multiples, max_multiples, companies_data['target_company'])

# Print estimated prices for target company
print_estimated_prices(mean_prices, "Mean")
print_estimated_prices(min_prices, "Min")
print_estimated_prices(max_prices, "Max")

# Estimate stock price range from multiples
min_price, mean_price, max_price = estimate_stock_price_from_range((mean_prices, min_prices, max_prices))

# Print estimated stock price range
print_estimated_stock_price_range(min_price, mean_price, max_price)


Company 1 Multiples:
P/Rev: 0.50
P/E: 7.03
P/B: 1.49
EV/EBIT: 16.25
EV/EBITDA: 11.64

Company 2 Multiples:
P/Rev: 0.57
P/E: 7.23
P/B: 1.24
EV/EBIT: 13.74
EV/EBITDA: 10.22

Mean Multiples:
P/Rev: 0.54
P/E: 7.13
P/B: 1.36
EV/EBIT: 14.99
EV/EBITDA: 10.93

Min Multiples:
P/Rev: 0.50
P/E: 7.03
P/B: 1.24
EV/EBIT: 13.74
EV/EBITDA: 10.22

Max Multiples:
P/Rev: 0.57
P/E: 7.23
P/B: 1.49
EV/EBIT: 16.25
EV/EBITDA: 11.64

Mean Prices for Target Company:
P/Rev: 216.88
P/E: 126.47
P/B: 244.91
EV/EBIT: 139.82
EV/EBITDA: 230.35

Min Prices for Target Company:
P/Rev: 202.63
P/E: 124.71
P/B: 221.93
EV/EBIT: 110.00
EV/EBITDA: 201.10

Max Prices for Target Company:
P/Rev: 231.14
P/E: 128.23
P/B: 267.89
EV/EBIT: 169.64
EV/EBITDA: 259.59

Estimated Stock Price for Target Company:
Min: 172.08, Mean: 191.69, Max: 211.30


In [15]:
def calculate_wacc(tau, re, rd, Ve, Vd):
    V = Ve + Vd
    WACC = (1 - tau) * rd * Vd / V + re * Ve / V
    return WACC

In [16]:
tau = .3
re = .1
rd = .05
Vd = 3
Ve = 5
calculate_wacc(tau, re, rd, Ve, Vd)

0.075625

In [ ]:
def black_scholes_valuation(asset_value, debt_value, risk_free_rate, volatility, time_to_maturity):
    """
    Calculate market value of equity and debt using the Black-Scholes model.
    
    Parameters:
    asset_value (float): Value of the firm's assets
    debt_value (float): Face value of debt
    risk_free_rate (float): Risk-free rate
    volatility (float): Volatility of the asset's value
    time_to_maturity (float): Time to maturity in years

    Returns:
    tuple: (Market value of equity, Market value of debt)
    """
    d1 = (np.log(asset_value / debt_value) + (risk_free_rate + 0.5 * volatility**2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
    d2 = d1 - volatility * np.sqrt(time_to_maturity)
    
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    
    equity_value = asset_value * N_d1 - debt_value * np.exp(-risk_free_rate * time_to_maturity) * N_d2
    debt_value = asset_value - equity_value
    
    return equity_value, debt_value

In [ ]:
asset_value = 100
debt_value = 50
risk_free_rate = 0.05
volatility = 0.2
time_to_maturity = 1

equity_value, debt_value = black_scholes_valuation(asset_value, debt_value, risk_free_rate, volatility, time_to_maturity)
print(f"Market value of equity: {equity_value:.2f}")
print(f"Market value of debt: {debt_value:.2f}")
